# Stocks

In [1]:
import requests
import pandas as pd
import json
import time
from datetime import date

In [ ]:
# Склад остатки - stocks?dateFrom=2023-04-12
# Заказы        - orders?dateFrom=2023-02-09&flag=0
# Продажи       - sales?dateFrom=2023-02-09&flag=0
# Поставки      - incomes?dateFrom=2023-03-19
# Отчет о продажах по реализации - reportDetailByPeriod?dateFrom=2023-02-09&dateTo=2023-02-28

In [2]:
# Выгружаем данные
# На выходе получается строка с информацие в виде словаря
url = "https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2023-01-01"
payload = {}
headers = {
  'Authorization': ''
}
response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)

[{"lastChangeDate":"2023-06-15T10:47:42.633","supplierArticle":"Этнический халат","techSize":"XS-М","barcode":"2037488839493","quantity":0,"isSupply":true,"isRealization":false,"quantityFull":0,"warehouseName":"Казань","nmId":147263932,"subject":"Халаты домашние","category":"Одежда","daysOnSite":113,"brand":"","SCCode":"Tech","Price":6400,"Discount":47},{"lastChangeDate":"2023-06-16T10:03:35","supplierArticle":"Этнический халат","techSize":"XS-М","barcode":"2037488839493","quantity":19,"isSupply":true,"isRealization":false,"quantityFull":22,"warehouseName":"Электросталь","nmId":147263932,"subject":"Халаты домашние","category":"Одежда","daysOnSite":115,"brand":"","SCCode":"Tech","Price":6400,"Discount":47},{"lastChangeDate":"2023-06-16T10:03:35","supplierArticle":"Шелковая пижама лист1","techSize":"XS-L","barcode":"2037488886244","quantity":9,"isSupply":true,"isRealization":false,"quantityFull":10,"warehouseName":"Коледино","nmId":147271346,"subject":"Пижамы","category":"Одежда","daysOn

In [3]:
# Данные в переменную
str_total = response.text  # достаём данные в строку
str_total

'[{"lastChangeDate":"2023-06-15T10:47:42.633","supplierArticle":"Этнический халат","techSize":"XS-М","barcode":"2037488839493","quantity":0,"isSupply":true,"isRealization":false,"quantityFull":0,"warehouseName":"Казань","nmId":147263932,"subject":"Халаты домашние","category":"Одежда","daysOnSite":113,"brand":"","SCCode":"Tech","Price":6400,"Discount":47},{"lastChangeDate":"2023-06-16T10:03:35","supplierArticle":"Этнический халат","techSize":"XS-М","barcode":"2037488839493","quantity":19,"isSupply":true,"isRealization":false,"quantityFull":22,"warehouseName":"Электросталь","nmId":147263932,"subject":"Халаты домашние","category":"Одежда","daysOnSite":115,"brand":"","SCCode":"Tech","Price":6400,"Discount":47},{"lastChangeDate":"2023-06-16T10:03:35","supplierArticle":"Шелковая пижама лист1","techSize":"XS-L","barcode":"2037488886244","quantity":9,"isSupply":true,"isRealization":false,"quantityFull":10,"warehouseName":"Коледино","nmId":147271346,"subject":"Пижамы","category":"Одежда","daysO

In [4]:
# Создаём первую строку датафрэйма
# Убираем лишние символы, разбиваем, берём первый заказ, добавляем символы для словаря
s_first = '{' + str_total.replace('[', '').replace(']', '').split('},{')[0].replace('{', '').replace('}', '') + '}'
# преобразуем словарь
json_acceptable_string = s_first.replace('"', "\"")
data = json.loads(json_acceptable_string)
# словарь в дф
df = pd.DataFrame([data])
df

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,warehouseName,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount
0,2023-06-15T10:47:42.633,Этнический халат,XS-М,2037488839493,0,True,False,0,Казань,147263932,Халаты домашние,Одежда,113,,Tech,6400,47


In [5]:
# Проходимся циклом по строке и дополняем датафрэйм
# Запускать 1 раз!
str_need = str_total.split('},{')
for i in str_need[1:]:  # Со второго элемента, так как первый уже закинул в дф
    i = '{' + i.replace('[', '').replace(']', '').replace('{', '').replace('}', '') + '}'
    json_acceptable_string = i.replace('"', "\"")
    data = json.loads(json_acceptable_string)
    df_t = pd.DataFrame([data])
    df = pd.concat([df, df_t])
df = df.reset_index(drop=True)
df.head(3)

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,warehouseName,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount
0,2023-06-15T10:47:42.633,Этнический халат,XS-М,2037488839493,0,True,False,0,Казань,147263932,Халаты домашние,Одежда,113,,Tech,6400,47
1,2023-06-16T10:03:35,Этнический халат,XS-М,2037488839493,19,True,False,22,Электросталь,147263932,Халаты домашние,Одежда,115,,Tech,6400,47
2,2023-06-16T10:03:35,Шелковая пижама лист1,XS-L,2037488886244,9,True,False,10,Коледино,147271346,Пижамы,Одежда,115,,Tech,6700,68


In [7]:
# Сохранили сегодняшней датой
today = date.today()
df.to_excel(f'stocks_{today}.xlsx', index=False)